<a href="https://colab.research.google.com/github/safin92/dz6_xor_voprosi/blob/main/XOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Задача XOR

# Задача: Реализация нейронной сети для решения проблемы XOR

## Введение

XOR (исключающее ИЛИ) - это логическая операция, которая возвращает истину только если входные значения различны. Таблица истинности для XOR:

| A | B | A XOR B |
|:-:|:-:|:-------:|
| 0 | 0 |    0    |
| 0 | 1 |    1    |
| 1 | 0 |    1    |
| 1 | 1 |    0    |

Эта задача не может быть решена с помощью линейной модели, поэтому она часто используется для демонстрации возможностей нейронных сетей.

## Задание

Ваша задача - реализовать двухслойную нейронную сеть для решения проблемы XOR.

### Архитектура сети:

- Входной слой: 2 нейрона (для A и B)
- Скрытый слой: 4 нейрона
- Выходной слой: 1 нейрон

![Архитектура нейронной сети для XOR](Tutorial-And.jpg)

## Инструкции:

1. Изучите предоставленный код класса `NeuralNetwork`.
2. Заполните все места, отмеченные `TODO`, используя предоставленные подсказки.
3. Используйте функции NumPy (`np.dot`, `np.random.uniform`, `np.sum`) для выполнения необходимых вычислений.
4. Реализуйте прямое и обратное распространение, а также обновление весов.
5. Обучите сеть на предоставленных данных XOR.
6. Протестируйте сеть и выведите результаты.

In [ ]:
import numpy as np

In [11]:
# Класс нейронной сети с одним скрытым слоем
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Инициализация параметров сети
        self.input_size = input_size      # Размер входного слоя (количество нейронов (признаков))
        self.hidden_size = hidden_size    # Количество нейронов в скрытом слое
        self.output_size = output_size    # Размер выходного слоя

        # Шаг 1: Инициализация весов и смещений
        # Весовые коэффициенты и смещения инициализируются случайным образом для разрыва симметрии в начальных значениях
        self.hidden_weights = np.random.uniform(size=(input_size, hidden_size)) # Веса между входным и скрытым слоем
        self.hidden_bias = np.random.uniform(size=(1, hidden_size))              # Смещения для скрытого слоя
        self.output_weights = np.random.uniform(size=(hidden_size, output_size)) # Веса между скрытым и выходным слоем
        self.output_bias = np.random.uniform(size=(1, output_size))              # Смещения для выходного слоя

    # Шаг 2: Функция активации - Сигмоида
    # Используется для нелинейности в нейронной сети
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # Шаг 3: Производная сигмоиды
    # Используется для вычисления градиентов при обратном распространении ошибки
    def sigmoid_derivative(self, x):
        return x * (1 - x)

    # Шаг 4: Прямое распространение (Forward Propagation)
    # Вычисление выходных значений сети на основе входных данных
    def forward(self, X):
        # Входной слой -> Скрытый слой
        # Применяем веса и смещения, затем функцию активации
        self.hidden_layer = self.sigmoid(np.dot(X, self.hidden_weights) + self.hidden_bias)

        # Скрытый слой -> Выходной слой
        # Применяем веса и смещения, затем функцию активации
        self.output_layer = self.sigmoid(np.dot(self.hidden_layer, self.output_weights) + self.output_bias)

        # Возвращаем выходное значение
        return self.output_layer

    # Шаг 5: Обратное распространение ошибки (Backward Propagation)
    # Обновление весов и смещений на основе ошибки предсказания
    def backward(self, X, y, output):
        # Шаг 5.1: Вычисление ошибки выходного слоя
        # Разница между фактическим и предсказанным значением
        error = y - output

        # Шаг 5.2: Вычисление градиента выходного слоя
        # Умножаем ошибку на производную функции активации
        d_output = error * self.sigmoid_derivative(output)

        # Шаг 5.3: Распространение ошибки на скрытый слой
        # Умножаем градиенты выходного слоя на веса выходного слоя
        error_hidden_layer = np.dot(d_output, self.output_weights.T)

        # Шаг 5.4: Вычисление градиента скрытого слоя
        # Умножаем ошибку скрытого слоя на производную функции активации
        d_hidden_layer = error_hidden_layer * self.sigmoid_derivative(self.hidden_layer)

        # Шаг 5.5: Обновление весов и смещений
        # Используем градиентный спуск для корректировки весов и смещений
        # Выходной слой
        self.output_weights += np.dot(self.hidden_layer.T, d_output) * self.learning_rate
        self.output_bias += np.sum(d_output, axis=0, keepdims=True) * self.learning_rate

        # Скрытый слой
        self.hidden_weights += np.dot(X.T, d_hidden_layer) * self.learning_rate
        self.hidden_bias += np.sum(d_hidden_layer, axis=0, keepdims=True) * self.learning_rate

    # Шаг 6: Обучение нейронной сети
    # Повторение этапов прямого и обратного распространения для каждой эпохи
    def train(self, X, y, epochs, learning_rate):
        self.learning_rate = learning_rate  # Устанавливаем скорость обучения
        for _ in range(epochs):
            output = self.forward(X)        # Прямое распространение
            self.backward(X, y, output)     # Обратное распространение

    # Шаг 7: Предсказание
    # Вычисление выхода для новых данных без обучения
    def predict(self, X):
        return self.forward(X)


In [12]:
# Наш "датасет" для обучения
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

In [13]:
# Инициализация нейронной сети
nn = NeuralNetwork(input_size=2, hidden_size=4, output_size=1)
#Цикл обучения
nn.train(X, y, epochs = 1000, learning_rate=10)

In [14]:
predictions = nn.predict(X)
print("Выходные данные после обучения:")
print(predictions)

# Подсказка: Используйте np.mean для вычисления среднего значения
# np.mean вычисляет среднее арифметическое элементов массива
accuracy = np.mean(np.round(predictions) == y)
print(f"\nТочность: {accuracy * 100}%")

Выходные данные после обучения:
[[0.99997724]
 [0.99999443]
 [0.99999296]
 [0.99999706]]

Точность: 50.0%


**1. Как называется алгоритм нейронный сетей.**

Градиентный спуск (или обртаное распространение ошибки). Элементы алгоритма включают в себя:
  
  1) Прямое распространение: данные проходят через сеть, от входного слоя к выходному - каждый нейрон взвешивает сумму входов, добавлет смещение и пропускает результат через функцию активации - на выходе сеть дает предсказание.
  
  2) Вычисление ошибки: сравниваем предсказание сети с правильным ответом и вычисляем ошибку (MSE, крос-энтропия).
  
  3) Обратное распространение ошибки: ошибка распространяется назад по сети, начиная с выходного слоя - вычисляются градиенты (производные функции по всем весам).
  
  4) Обновление весов: градиентный спуск корректирует веса так, чтобы уменьшить ошибку - вес обновляется по формуле (новый вес = старый вес - скорость обучения*градиент ошибки по весу.


**2. По какому методу градиентного спуска реализован алгоритм?**

Методы градиентного спуска:
  
  1) Полный (Batch) градиентный спуск (Batch Gradient Descent, BGD)

Обновляет веса раз в эпоху (полный проход посети), используя всю выборку данных.
Точный, но медленный и требует много памяти.
Хорош для гладких функций (например, в линейной регрессии).

  2) Стохастический градиентный спуск (Stochastic Gradient Descent, SGD)

Обновляет веса после каждого примера (пример - отдельный вход, один объект данных , который подается на вход нейросети).
Быстрее, но из-за случайности обновлений может колебаться и не всегда сходится к оптимальному минимуму.
Хорош для онлайн-обучения.

  3) Мини-батч градиентный спуск (Mini-Batch Gradient Descent, MBGD)

Компромисс между BGD и SGD. Разбивает выборку на маленькие группы (батчи) и обновляет веса после обработки каждого батча.
Быстрее, чем BGD, и менее шумный, чем SGD.

  4) Adagrad (Adaptive Gradient Descent)

Разные параметры обучаются с разной скоростью.
Хорошо работает для разреженных данных.
Проблема: скорость обучения со временем становится слишком маленькой.

  5) RMSprop (Root Mean Square Propagation)

Улучшает Adagrad: добавляет затухающее среднее квадратов градиентов.
Хорошо работает в глубоких сетях.

  6) Adam (Adaptive Moment Estimation) – самый популярный

Комбинирует Momentum и RMSprop.
Самый распространенный метод в нейросетях.
Автоматически регулирует скорость обучения для каждого параметра.

Какой метод выбрать?
Если данных мало → BGD (полный градиентный спуск).
Если данных много → Mini-Batch или Adam.
Для онлайн-обучения → SGD.
Для глубоких сетей → Adam или RMSprop.

В коде реализован полный (Batch) градиентный спуск, так как веса обновляются после обработки всего набора данных.

**3. Как подбирать Learning_rate**

Скорость обучения - коэффициент, который определяет, насколько сильно обновляются веса нейросети на каждом шаге обучения.

Выбор learning_rate критически важен:

Слишком маленький α → обучение идет очень медленно, можно застрять в локальном минимуме.
Слишком большой α → веса обновляются слишком резко, может начаться "скачкообразное" обучение или сеть вообще не сойдется.

Методы определения коэффициента:

1) Метод проб и ошибок (эмпирический поиск)
Простейший способ — начать с небольшого значения (0.01 или 0.001) и наблюдать за изменением ошибки.

Если ошибка уменьшается медленно → попробуй увеличить α.
Если ошибка скачет или растет → уменьшай α.

Пример:

nn.train(X, y, epochs=1000, learning_rate=0.01)

Начни с 0.01. Если градиент "скачет" → уменьши α (0.001).
Если сходится медленно → попробуй 0.05 или 0.1.

2) Метод логарифмического поиска (поиск по степеням 10)
Попробуй разные значения α в диапазоне от 1e-5 до 1e-1 (от 0.00001 до 0.1):

    for lr in [0.00001, 0.0001, 0.001, 0.01, 0.1]:

    print(f"Тестируем learning_rate = {lr}")

    nn.train(X, y, epochs=1000, learning_rate=lr)
    
Если α = 0.1 работает хорошо, попробуй 0.05 или 0.2 для уточнения.

3) Адаптивные методы (Adam, RMSprop, Adagrad)
Можно использовать автоматическую подстройку learning_rate:

Adam — один из лучших оптимизаторов (сочетает Momentum + RMSprop).
RMSprop — хорошо работает на сложных задачах.
Adagrad — подходит, если данные сильно различаются по масштабам.

Пример с Adam в Keras:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.01)

4) Эксперимент с "learning rate decay" (уменьшение скорости со временем)
Иногда обучение сначала требует большого α, но потом его лучше уменьшить. Это называется "learning rate decay" (затухание скорости обучения).

Пример:

    initial_lr = 0.1
    decay_rate = 0.01
    lr = initial_lr / (1 + decay_rate * epoch)  # Уменьшаем lr по мере роста эпох

Чем дальше обучение, тем меньше шаги обновления весов, что помогает точнее настроить сеть.

5) Визуальный анализ (график ошибки)
Можно строить график ошибки (loss function) в зависимости от эпох.

Пример:

    import matplotlib.pyplot as plt
    
    losses = []
    for epoch in range(1000):
      output = nn.forward(X)
      loss = np.mean((y - output) ** 2)  # Среднеквадратичная ошибка (MSE)
      losses.append(loss)
    nn.backward(X, y, output)

    plt.plot(losses)
    plt.xlabel("Эпохи")
    plt.ylabel("Ошибка (loss)")
    plt.show()

Если кривая падает медленно → увеличь α.
Если скачет или растет → уменьши α.

Вывод:
Начинай с α = 0.01 или 0.001, тестируй.
Используй логарифмический поиск (0.00001 → 0.0001 → 0.001 → 0.01 → 0.1).
Для сложных моделей попробуй Adam или RMSprop вместо стандартного градиентного спуска.
Можно уменьшать α со временем (learning rate decay).
Строй графики ошибки, чтобы видеть, как α влияет на обучение.
Лучший learning_rate зависит от данных! 🚀

**4. Что такое np.random.uniform?**

np.random — это модуль в NumPy, который содержит множество функций для генерации случайных чисел.

np.random.uniform(low, high, size) — это конкретная функция внутри np.random, которая генерирует равномерно распределенные числа в заданном диапазоне [low, high].

**5) веса задать нулями и смешения тоже**

Ниже представлен код с начальными весами и смещениями равными 0.


In [10]:
import numpy as np
# Класс нейронной сети с одним скрытым слоем
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Инициализация параметров сети
        self.input_size = input_size      # Размер входного слоя (количество нейронов (признаков))
        self.hidden_size = hidden_size    # Количество нейронов в скрытом слое
        self.output_size = output_size    # Размер выходного слоя

        # Шаг 1: Инициализация весов и смещений
        # Весовые коэффициенты и смещения инициализируются случайным образом для разрыва симметрии в начальных значениях
        self.hidden_weights = np.zeros((input_size, hidden_size)) # Веса между входным и скрытым слоем = 0
        self.hidden_bias = np.zeros((1, hidden_size))              # Смещения для скрытого слоя = 0
        self.output_weights = np.zeros((hidden_size, output_size)) # Веса между скрытым и выходным слоем = 0
        self.output_bias = np.zeros((1, output_size))              # Смещения для выходного слоя = 0

    # Шаг 2: Функция активации - Сигмоида
    # Используется для нелинейности в нейронной сети
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # Шаг 3: Производная сигмоиды
    # Используется для вычисления градиентов при обратном распространении ошибки
    def sigmoid_derivative(self, x):
        return x * (1 - x)

    # Шаг 4: Прямое распространение (Forward Propagation)
    # Вычисление выходных значений сети на основе входных данных
    def forward(self, X):
        # Входной слой -> Скрытый слой
        # Применяем веса и смещения, затем функцию активации
        self.hidden_layer = self.sigmoid(np.dot(X, self.hidden_weights) + self.hidden_bias)

        # Скрытый слой -> Выходной слой
        # Применяем веса и смещения, затем функцию активации
        self.output_layer = self.sigmoid(np.dot(self.hidden_layer, self.output_weights) + self.output_bias)

        # Возвращаем выходное значение
        return self.output_layer

    # Шаг 5: Обратное распространение ошибки (Backward Propagation)
    # Обновление весов и смещений на основе ошибки предсказания
    def backward(self, X, y, output):
        # Шаг 5.1: Вычисление ошибки выходного слоя
        # Разница между фактическим и предсказанным значением
        error = y - output

        # Шаг 5.2: Вычисление градиента выходного слоя
        # Умножаем ошибку на производную функции активации
        d_output = error * self.sigmoid_derivative(output)

        # Шаг 5.3: Распространение ошибки на скрытый слой
        # Умножаем градиенты выходного слоя на веса выходного слоя
        error_hidden_layer = np.dot(d_output, self.output_weights.T)

        # Шаг 5.4: Вычисление градиента скрытого слоя
        # Умножаем ошибку скрытого слоя на производную функции активации
        d_hidden_layer = error_hidden_layer * self.sigmoid_derivative(self.hidden_layer)

        # Шаг 5.5: Обновление весов и смещений
        # Используем градиентный спуск для корректировки весов и смещений
        # Выходной слой
        self.output_weights += np.dot(self.hidden_layer.T, d_output) * self.learning_rate
        self.output_bias += np.sum(d_output, axis=0, keepdims=True) * self.learning_rate

        # Скрытый слой
        self.hidden_weights += np.dot(X.T, d_hidden_layer) * self.learning_rate
        self.hidden_bias += np.sum(d_hidden_layer, axis=0, keepdims=True) * self.learning_rate

    # Шаг 6: Обучение нейронной сети
    # Повторение этапов прямого и обратного распространения для каждой эпохи
    def train(self, X, y, epochs, learning_rate):
        self.learning_rate = learning_rate  # Устанавливаем скорость обучения
        for _ in range(epochs):
            output = self.forward(X)        # Прямое распространение
            self.backward(X, y, output)     # Обратное распространение

    # Шаг 7: Предсказание
    # Вычисление выхода для новых данных без обучения
    def predict(self, X):
        return self.forward(X)

# Наш "датасет" для обучения
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

# Инициализация нейронной сети
nn = NeuralNetwork(input_size=2, hidden_size=4, output_size=1)
#Цикл обучения
nn.train(X, y, epochs = 1000, learning_rate=10)

predictions = nn.predict(X)
print("Выходные данные после обучения:")
print(predictions)

# Подсказка: Используйте np.mean для вычисления среднего значения
# np.mean вычисляет среднее арифметическое элементов массива
accuracy = np.mean(np.round(predictions) == y)
print(f"\nТочность: {accuracy * 100}%")

Выходные данные после обучения:
[[0.5]
 [0.5]
 [0.5]
 [0.5]]

Точность: 50.0%


При задании различных смещений нейросеть продолжает корректную раьботу, если задать и веса и смещения нулями, то нейросеть не обучается.

1) Все нейроны в одном слое получают одинаковые градиенты.
Во время обратного распространения (backpropagation) градиенты вычисляются через веса. Если все веса изначально нули, то градиенты для всех нейронов будут одинаковыми.
Это приводит к разрыву симметрии — все нейроны обновляются одинаково, и сеть не может учиться.

2) Сеть остаётся линейной.
Без случайной инициализации веса остаются одинаковыми, а значит, каждый слой выполняет линейное преобразование.
Линейная модель не способна обучаться сложным зависимостям (а нейросеть нужна именно для нелинейного обучения).

**6. В шаге 5.5 используется "+=", почему не "-+"?**

Классическая формула обновления весов выглядит так:

𝑊 = 𝑊 − 𝜂⋅∂𝐿

где:

𝑊 — веса,
𝜂 (learning rate) — скорость обучения,
∂𝐿 — градиент функции ошибки (L — loss).
Таким образом, обычно мы вычитаем градиент (-=), чтобы двигаться в сторону убывания ошибки.

Правильно использовать -=, но в коде уже имеется ошибка, которая позволяет нам использовать плюс - error = y - output. Чтобы использовать -=, необходимо заменить формулу error = y - output на error = output - y



In [16]:
import numpy as np
# Класс нейронной сети с одним скрытым слоем
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Инициализация параметров сети
        self.input_size = input_size      # Размер входного слоя (количество нейронов (признаков))
        self.hidden_size = hidden_size    # Количество нейронов в скрытом слое
        self.output_size = output_size    # Размер выходного слоя

        # Шаг 1: Инициализация весов и смещений
        # Весовые коэффициенты и смещения инициализируются случайным образом для разрыва симметрии в начальных значениях
        self.hidden_weights = np.random.uniform(size = (input_size, hidden_size)) # Веса между входным и скрытым слоем
        self.hidden_bias = np.random.uniform(size = (1, hidden_size))              # Смещения для скрытого слоя
        self.output_weights = np.random.uniform(size = (hidden_size, output_size)) # Веса между скрытым и выходным слоем
        self.output_bias = np.random.uniform(size = (1, output_size))              # Смещения для выходного слоя

    # Шаг 2: Функция активации - Сигмоида
    # Используется для нелинейности в нейронной сети
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # Шаг 3: Производная сигмоиды
    # Используется для вычисления градиентов при обратном распространении ошибки
    def sigmoid_derivative(self, x):
        return x * (1 - x)

    # Шаг 4: Прямое распространение (Forward Propagation)
    # Вычисление выходных значений сети на основе входных данных
    def forward(self, X):
        # Входной слой -> Скрытый слой
        # Применяем веса и смещения, затем функцию активации
        self.hidden_layer = self.sigmoid(np.dot(X, self.hidden_weights) + self.hidden_bias)

        # Скрытый слой -> Выходной слой
        # Применяем веса и смещения, затем функцию активации
        self.output_layer = self.sigmoid(np.dot(self.hidden_layer, self.output_weights) + self.output_bias)

        # Возвращаем выходное значение
        return self.output_layer

    # Шаг 5: Обратное распространение ошибки (Backward Propagation)
    # Обновление весов и смещений на основе ошибки предсказания
    def backward(self, X, y, output):
        # Шаг 5.1: Вычисление ошибки выходного слоя
        # Разница между фактическим и предсказанным значением
        error = output - y

        # Шаг 5.2: Вычисление градиента выходного слоя
        # Умножаем ошибку на производную функции активации
        d_output = error * self.sigmoid_derivative(output)

        # Шаг 5.3: Распространение ошибки на скрытый слой
        # Умножаем градиенты выходного слоя на веса выходного слоя
        error_hidden_layer = np.dot(d_output, self.output_weights.T)

        # Шаг 5.4: Вычисление градиента скрытого слоя
        # Умножаем ошибку скрытого слоя на производную функции активации
        d_hidden_layer = error_hidden_layer * self.sigmoid_derivative(self.hidden_layer)

        # Шаг 5.5: Обновление весов и смещений
        # Используем градиентный спуск для корректировки весов и смещений
        # Выходной слой
        self.output_weights -= np.dot(self.hidden_layer.T, d_output) * self.learning_rate
        self.output_bias -= np.sum(d_output, axis=0, keepdims=True) * self.learning_rate

        # Скрытый слой
        self.hidden_weights -= np.dot(X.T, d_hidden_layer) * self.learning_rate
        self.hidden_bias -= np.sum(d_hidden_layer, axis=0, keepdims=True) * self.learning_rate

    # Шаг 6: Обучение нейронной сети
    # Повторение этапов прямого и обратного распространения для каждой эпохи
    def train(self, X, y, epochs, learning_rate):
        self.learning_rate = learning_rate  # Устанавливаем скорость обучения
        for _ in range(epochs):
            output = self.forward(X)        # Прямое распространение
            self.backward(X, y, output)     # Обратное распространение

    # Шаг 7: Предсказание
    # Вычисление выхода для новых данных без обучения
    def predict(self, X):
        return self.forward(X)

# Наш "датасет" для обучения
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

# Инициализация нейронной сети
nn = NeuralNetwork(input_size=2, hidden_size=4, output_size=1)
#Цикл обучения
nn.train(X, y, epochs = 1000, learning_rate=10)

predictions = nn.predict(X)
print("Выходные данные после обучения:")
print(predictions)

# Подсказка: Используйте np.mean для вычисления среднего значения
# np.mean вычисляет среднее арифметическое элементов массива
accuracy = np.mean(np.round(predictions) == y)
print(f"\nТочность: {accuracy * 100}%")

Выходные данные после обучения:
[[0.00979599]
 [0.98621319]
 [0.99202569]
 [0.01480533]]

Точность: 100.0%


**7. Попробовать софтмакс вместо сигмоиды, обучить софтмаксом**

Вариант полного использования Softmax

In [38]:
import numpy as np

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Инициализация параметров сети
        self.input_size = input_size      # Размер входного слоя
        self.hidden_size = hidden_size    # Количество нейронов в скрытом слое
        self.output_size = output_size    # Размер выходного слоя

        # Инициализация весов и смещений
        self.hidden_weights = np.random.uniform(size=(input_size, hidden_size))  # Веса между входным и скрытым слоем
        self.hidden_bias = np.random.uniform(size=(1, hidden_size))              # Смещения для скрытого слоя
        self.output_weights = np.random.uniform(size=(hidden_size, output_size)) # Веса между скрытым и выходным слоем
        self.output_bias = np.random.uniform(size=(1, output_size))              # Смещения для выходного слоя

    # Функция активации - Softmax
    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # Нормализация для стабильности
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    # Производная Softmax
    def softmax_derivative(self, x):
        s = x.reshape(-1, 1)
        return np.diagflat(s) - np.dot(s, s.T)

    # Прямое распространение (Forward Propagation)
    def forward(self, X):
        # Входной слой -> Скрытый слой
        self.hidden_layer_input = np.dot(X, self.hidden_weights) + self.hidden_bias
        self.hidden_layer_output = self.softmax(self.hidden_layer_input)

        # Скрытый слой -> Выходной слой
        self.output_layer_input = np.dot(self.hidden_layer_output, self.output_weights) + self.output_bias
        self.output_layer_output = self.softmax(self.output_layer_input)

        return self.output_layer_output

    # Обратное распространение ошибки (Backward Propagation)
    def backward(self, X, y, output):
        batch_size = X.shape[0]

        # Ошибка выходного слоя
        error = output - y

        # Градиент выходного слоя
        d_output = np.zeros_like(output)
        for i in range(batch_size):
            d_output[i] = np.dot(self.softmax_derivative(output[i]), error[i])

        # Ошибка скрытого слоя
        error_hidden_layer = np.dot(d_output, self.output_weights.T)

        # Градиент скрытого слоя
        d_hidden_layer = np.zeros_like(self.hidden_layer_output)
        for i in range(batch_size):
            d_hidden_layer[i] = np.dot(self.softmax_derivative(self.hidden_layer_output[i]), error_hidden_layer[i])

        # Обновление весов и смещений
        self.output_weights -= np.dot(self.hidden_layer_output.T, d_output) * self.learning_rate / batch_size
        self.output_bias -= np.sum(d_output, axis=0, keepdims=True) * self.learning_rate / batch_size

        self.hidden_weights -= np.dot(X.T, d_hidden_layer) * self.learning_rate / batch_size
        self.hidden_bias -= np.sum(d_hidden_layer, axis=0, keepdims=True) * self.learning_rate / batch_size

    # Обучение нейронной сети
    def train(self, X, y, epochs, learning_rate):
        self.learning_rate = learning_rate  # Устанавливаем скорость обучения
        for _ in range(epochs):
            output = self.forward(X)        # Прямое распространение
            self.backward(X, y, output)     # Обратное распространение

    # Предсказание
    def predict(self, X):
        return self.forward(X)

# Датасет для обучения
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[1, 0], [0, 1], [0, 1], [1, 0]])  # One-hot encoding для многоклассовой классификации

# Инициализация нейронной сети
nn = NeuralNetwork(input_size=2, hidden_size=4, output_size=2)

# Цикл обучения
nn.train(X, y, epochs=1000, learning_rate=10)

# Предсказание
predictions = nn.predict(X)
print("Выходные данные после обучения:")
print(predictions)

# Точность
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y, axis=1)
accuracy = np.mean(predicted_classes == true_classes)
print(f"\nТочность: {accuracy * 100}%")

Выходные данные после обучения:
[[0.99344673 0.00655327]
 [0.00792608 0.99207392]
 [0.00782483 0.99217517]
 [0.99117347 0.00882653]]

Точность: 100.0%


Softmax на выходном слое

In [44]:
import numpy as np

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Инициализация параметров сети
        self.input_size = input_size      # Размер входного слоя
        self.hidden_size = hidden_size    # Количество нейронов в скрытом слое
        self.output_size = output_size    # Размер выходного слоя

        # Инициализация весов и смещений
        self.hidden_weights = np.random.uniform(size=(input_size, hidden_size))  # Веса между входным и скрытым слоем
        self.hidden_bias = np.random.uniform(size=(1, hidden_size))              # Смещения для скрытого слоя
        self.output_weights = np.random.uniform(size=(hidden_size, output_size)) # Веса между скрытым и выходным слоем
        self.output_bias = np.random.uniform(size=(1, output_size))              # Смещения для выходного слоя

    # Функция активации - Softmax
    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # Нормализация для стабильности
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    # Функция активации - Сигмоида
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # Производная сигмоиды
    def sigmoid_derivative(self, x):
        return x * (1 - x)

    # Прямое распространение (Forward Propagation)
    def forward(self, X):
        # Входной слой -> Скрытый слой
        self.hidden_layer_input = np.dot(X, self.hidden_weights) + self.hidden_bias
        self.hidden_layer_output = self.sigmoid(self.hidden_layer_input)

        # Скрытый слой -> Выходной слой
        self.output_layer_input = np.dot(self.hidden_layer_output, self.output_weights) + self.output_bias
        self.output_layer_output = self.softmax(self.output_layer_input)

        return self.output_layer_output

    # Обратное распространение ошибки (Backward Propagation)
    def backward(self, X, y, output):
        batch_size = X.shape[0]

        # Ошибка выходного слоя
        error = output - y

        # Градиент выходного слоя
        d_output = error  # Для Softmax градиент равен разнице между предсказанием и целевым значением

        # Ошибка скрытого слоя
        error_hidden_layer = np.dot(d_output, self.output_weights.T)

        # Градиент скрытого слоя
        d_hidden_layer = error_hidden_layer * self.sigmoid_derivative(self.hidden_layer_output)

        # Обновление весов и смещений
        self.output_weights -= np.dot(self.hidden_layer_output.T, d_output) * self.learning_rate / batch_size
        self.output_bias -= np.sum(d_output, axis=0, keepdims=True) * self.learning_rate / batch_size

        self.hidden_weights -= np.dot(X.T, d_hidden_layer) * self.learning_rate / batch_size
        self.hidden_bias -= np.sum(d_hidden_layer, axis=0, keepdims=True) * self.learning_rate / batch_size

    # Обучение нейронной сети
    def train(self, X, y, epochs, learning_rate):
        self.learning_rate = learning_rate  # Устанавливаем скорость обучения
        for _ in range(epochs):
            output = self.forward(X)        # Прямое распространение
            self.backward(X, y, output)     # Обратное распространение

    # Предсказание
    def predict(self, X):
        return self.forward(X)

# Датасет для обучения
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[1, 0], [0, 1], [0, 1], [1, 0]])  # One-hot encoding для многоклассовой классификации

# Инициализация нейронной сети
nn = NeuralNetwork(input_size=2, hidden_size=4, output_size=2)

# Цикл обучения
nn.train(X, y, epochs=1000, learning_rate=10)

# Предсказание
predictions = nn.predict(X)
print("Выходные данные после обучения:")
print(predictions)

# Точность
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y, axis=1)
accuracy = np.mean(predicted_classes == true_classes)
print(f"\nТочность: {accuracy * 100}%")

Выходные данные после обучения:
[[9.99177783e-01 8.22216852e-04]
 [5.13253049e-04 9.99486747e-01]
 [6.44733945e-04 9.99355266e-01]
 [9.99374725e-01 6.25274536e-04]]

Точность: 100.0%


Сравнение всех вариантов по точности и производительности

In [79]:
import numpy as np
import time

# Класс с Softmax на всех слоях
class NeuralNetworkSoftmaxAllLayers:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.hidden_weights = np.random.uniform(size=(input_size, hidden_size))
        self.hidden_bias = np.random.uniform(size=(1, hidden_size))
        self.output_weights = np.random.uniform(size=(hidden_size, output_size))
        self.output_bias = np.random.uniform(size=(1, output_size))

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def softmax_derivative(self, x):
        s = x.reshape(-1, 1)
        return np.diagflat(s) - np.dot(s, s.T)

    def forward(self, X):
        self.hidden_layer_input = np.dot(X, self.hidden_weights) + self.hidden_bias
        self.hidden_layer_output = self.softmax(self.hidden_layer_input)

        self.output_layer_input = np.dot(self.hidden_layer_output, self.output_weights) + self.output_bias
        self.output_layer_output = self.softmax(self.output_layer_input)
        return self.output_layer_output

    def backward(self, X, y, output):
        batch_size = X.shape[0]
        error = output - y
        d_output = error

        # Градиент для выходного слоя
        error_hidden_layer = np.dot(d_output, self.output_weights.T)

        # Градиент для скрытого слоя (вычисляем для каждого примера отдельно)
        d_hidden_layer = np.zeros_like(self.hidden_layer_output)
        for i in range(batch_size):
            softmax_deriv = self.softmax_derivative(self.hidden_layer_output[i])
            d_hidden_layer[i] = np.dot(error_hidden_layer[i], softmax_deriv)

        # Обновление весов и смещений
        self.output_weights -= np.dot(self.hidden_layer_output.T, d_output) * self.learning_rate / batch_size
        self.output_bias -= np.sum(d_output, axis=0, keepdims=True) * self.learning_rate / batch_size

        self.hidden_weights -= np.dot(X.T, d_hidden_layer) * self.learning_rate / batch_size
        self.hidden_bias -= np.sum(d_hidden_layer, axis=0, keepdims=True) * self.learning_rate / batch_size

    def train(self, X, y, epochs, learning_rate):
        self.learning_rate = learning_rate
        for _ in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output)

    def predict(self, X):
        return self.forward(X)


# Класс с Softmax только на выходном слое
class NeuralNetworkSoftmaxOutputLayer:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.hidden_weights = np.random.uniform(size=(input_size, hidden_size))
        self.hidden_bias = np.random.uniform(size=(1, hidden_size))
        self.output_weights = np.random.uniform(size=(hidden_size, output_size))
        self.output_bias = np.random.uniform(size=(1, output_size))

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, X):
        self.hidden_layer_input = np.dot(X, self.hidden_weights) + self.hidden_bias
        self.hidden_layer_output = self.sigmoid(self.hidden_layer_input)

        self.output_layer_input = np.dot(self.hidden_layer_output, self.output_weights) + self.output_bias
        self.output_layer_output = self.softmax(self.output_layer_input)
        return self.output_layer_output

    def backward(self, X, y, output):
        batch_size = X.shape[0]
        error = output - y
        d_output = error

        error_hidden_layer = np.dot(d_output, self.output_weights.T)
        d_hidden_layer = error_hidden_layer * self.sigmoid_derivative(self.hidden_layer_output)

        self.output_weights -= np.dot(self.hidden_layer_output.T, d_output) * self.learning_rate / batch_size
        self.output_bias -= np.sum(d_output, axis=0, keepdims=True) * self.learning_rate / batch_size

        self.hidden_weights -= np.dot(X.T, d_hidden_layer) * self.learning_rate / batch_size
        self.hidden_bias -= np.sum(d_hidden_layer, axis=0, keepdims=True) * self.learning_rate / batch_size

    def train(self, X, y, epochs, learning_rate):
        self.learning_rate = learning_rate
        for _ in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output)

    def predict(self, X):
        return self.forward(X)


# Класс с сигмоидой на всех слоях
class NeuralNetworkSigmoidAllLayers:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.hidden_weights = np.random.uniform(size=(input_size, hidden_size))
        self.hidden_bias = np.random.uniform(size=(1, hidden_size))
        self.output_weights = np.random.uniform(size=(hidden_size, output_size))
        self.output_bias = np.random.uniform(size=(1, output_size))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, X):
        self.hidden_layer_input = np.dot(X, self.hidden_weights) + self.hidden_bias
        self.hidden_layer_output = self.sigmoid(self.hidden_layer_input)

        self.output_layer_input = np.dot(self.hidden_layer_output, self.output_weights) + self.output_bias
        self.output_layer_output = self.sigmoid(self.output_layer_input)
        return self.output_layer_output

    def backward(self, X, y, output):
        batch_size = X.shape[0]
        error = output - y
        d_output = error * self.sigmoid_derivative(output)

        error_hidden_layer = np.dot(d_output, self.output_weights.T)
        d_hidden_layer = error_hidden_layer * self.sigmoid_derivative(self.hidden_layer_output)

        self.output_weights -= np.dot(self.hidden_layer_output.T, d_output) * self.learning_rate / batch_size
        self.output_bias -= np.sum(d_output, axis=0, keepdims=True) * self.learning_rate / batch_size

        self.hidden_weights -= np.dot(X.T, d_hidden_layer) * self.learning_rate / batch_size
        self.hidden_bias -= np.sum(d_hidden_layer, axis=0, keepdims=True) * self.learning_rate / batch_size

    def train(self, X, y, epochs, learning_rate):
        self.learning_rate = learning_rate
        for _ in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output)

    def predict(self, X):
        return self.forward(X)


# Датасет для обучения
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[1, 0], [0, 1], [0, 1], [1, 0]])  # One-hot encoding

# Параметры обучения
epochs = 1000
learning_rate = 10

# Создание сетей
nn_softmax_all = NeuralNetworkSoftmaxAllLayers(input_size=2, hidden_size=4, output_size=2)
nn_softmax_output = NeuralNetworkSoftmaxOutputLayer(input_size=2, hidden_size=4, output_size=2)
nn_sigmoid_all = NeuralNetworkSigmoidAllLayers(input_size=2, hidden_size=4, output_size=2)

# Обучение и измерение времени
start_time = time.time()
nn_softmax_all.train(X, y, epochs, learning_rate)
time_softmax_all = time.time() - start_time

start_time = time.time()
nn_softmax_output.train(X, y, epochs, learning_rate)
time_softmax_output = time.time() - start_time

start_time = time.time()
nn_sigmoid_all.train(X, y, epochs, learning_rate)
time_sigmoid_all = time.time() - start_time

# Предсказание и точность
predictions_all = nn_softmax_all.predict(X)
predictions_output = nn_softmax_output.predict(X)
predictions_sigmoid_all = nn_sigmoid_all.predict(X)

accuracy_all = np.mean(np.argmax(predictions_all, axis=1) == np.argmax(y, axis=1))
accuracy_output = np.mean(np.argmax(predictions_output, axis=1) == np.argmax(y, axis=1))
accuracy_sigmoid_all = np.mean(np.argmax(predictions_sigmoid_all, axis=1) == np.argmax(y, axis=1))

# Вывод результатов
print(f"Точность (Softmax на всех слоях): {accuracy_all * 100:.2f}%")
print(f"Точность (Softmax только на выходном слое): {accuracy_output * 100:.2f}%")
print(f"Точность (Сигмоида на всех слоях): {accuracy_sigmoid_all * 100:.2f}%")
print(f"Время обучения (Softmax на всех слоях): {time_softmax_all:.4f} сек")
print(f"Время обучения (Softmax только на выходном слое): {time_softmax_output:.4f} сек")
print(f"Время обучения (Сигмоида на всех слоях): {time_sigmoid_all:.4f} сек")

Точность (Softmax на всех слоях): 75.00%
Точность (Softmax только на выходном слое): 75.00%
Точность (Сигмоида на всех слоях): 100.00%
Время обучения (Softmax на всех слоях): 0.0996 сек
Время обучения (Softmax только на выходном слое): 0.0531 сек
Время обучения (Сигмоида на всех слоях): 0.0445 сек


Результаты по точности и производительности всегда разные при различных величинах эпох и скорости обучения.
Softmax применяется только выходном слое, если использовать на всех слоях, то можно столкнуться с замедлением обучения, нестабильностью градиентов и ухудшением качества модели.

